In [16]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Parameters for data generation
num_points = 50  # Number of grid points
lat_range = (-90, 90)  # Latitude range
lon_range = (-180, 180)  # Longitude range
temperature_range = (10, 30)  # Temperature range (°C)
salinity_range = (30, 40)  # Salinity range (PSU)
time_steps = 10  # Number of time steps per grid point
start_date = datetime(2024, 1, 1)

# Generate random data
data = []
for _ in range(num_points):
    lat = round(random.uniform(*lat_range), 2)  # Random latitude
    lon = round(random.uniform(*lon_range), 2)  # Random longitude
    for step in range(time_steps):
        date = start_date + timedelta(days=step)
        temperature = round(random.uniform(*temperature_range), 2)  # Random temperature
        salinity = round(random.uniform(*salinity_range), 2)  # Random salinity
        data.append([lat, lon, date, temperature, salinity])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Latitude", "Longitude", "Date", "Temperature (°C)", "Salinity (PSU)"])

# Save the data to a CSV file
df.to_csv("grid_temperature_salinity_timeseries.csv", index=False)

print("Time-series data saved to 'grid_temperature_salinity_timeseries.csv'")


Time-series data saved to 'grid_temperature_salinity_timeseries.csv'


In [15]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [18]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the time-series gridpoint data
data = pd.read_csv("grid_temperature_salinity_timeseries.csv")
data['Date'] = pd.to_datetime(data['Date'])

# Initialize the Dash app
app = dash.Dash(__name__)

# App Layout
app.layout = html.Div([
    html.H1("Temperature and Salinity Grid Dashboard", style={'textAlign': 'center'}),

    # Scatter mapbox plot
    html.Label("Click a grid point on the map to see time series:"),
    dcc.Graph(id='grid-map'),

    # Time series plots
    html.Div([
        html.Div([
            dcc.Graph(id='temperature-time-series')
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='salinity-time-series')
        ], style={'width': '48%', 'display': 'inline-block'}),
    ])
])

# Callback to update the scatter map
@app.callback(
    Output('grid-map', 'figure'),
    [Input('grid-map', 'id')]  # Use 'id' as the trigger to initialize the map
)
def update_map(_):
    # Create a scatter_geo map
    fig = px.scatter_geo(
        data.groupby(['Latitude', 'Longitude'])[['Temperature (°C)', 'Salinity (PSU)']].mean().reset_index(),
        lat="Latitude",
        lon="Longitude",
        size="Temperature (°C)",  # Optional: size the points by temperature
        color="Temperature (°C)",  # Optional: color the points by temperature
        title="Grid Points - Temperature (°C)",
        color_continuous_scale="Viridis",
        projection="natural earth",
    )
    return fig

# Callback to update the time-series plots
@app.callback(
    [Output('temperature-time-series', 'figure'),
     Output('salinity-time-series', 'figure')],
    [Input('grid-map', 'clickData')]  # Listen for click events on the map
)
def update_time_series(click_data):
    # If no grid point is clicked, return empty figures
    if not click_data:
        return {}, {}

    # Extract latitude and longitude of clicked point
    lat = click_data['points'][0]['lat']
    lon = click_data['points'][0]['lon']

    # Filter data for the selected grid point
    filtered_data = data[(data['Latitude'] == lat) & (data['Longitude'] == lon)]

    # Create time-series plot for temperature
    temp_fig = px.line(
        filtered_data,
        x='Date',
        y='Temperature (°C)',
        title=f"Temperature Time Series at ({lat}, {lon})",
        labels={'Date': 'Date', 'Temperature (°C)': 'Temperature (°C)'},
        color_discrete_sequence=['orange']
    )

    # Create time-series plot for salinity
    salinity_fig = px.line(
        filtered_data,
        x='Date',
        y='Salinity (PSU)',
        title=f"Salinity Time Series at ({lat}, {lon})",
        labels={'Date': 'Date', 'Salinity (PSU)': 'Salinity (PSU)'},
        color_discrete_sequence=['blue']
    )

    return temp_fig, salinity_fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
